# Notebook Programación: Data Cleaning

## Importo librerias

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
import seaborn as sns

# Cargo el dataset original del Banco Mundial

In [2]:
df = pd.read_csv(r"C:\Users\Victor\Desktop\EDA_Victor_Bandin\src\data\FinStructure_2012_September_Update2x.csv")

# Limpio el dataset para poder trabajar con él

In [3]:
df=df.fillna('Latin America & the Caribbean')
fields = ['dbacba','llgdp','cbagdp','dbagdp','ofagdp','pcrdbgdp','pcrdbofgdp','bdgdp',
          'fdgdp','bcbd','ll_usd','overhead','netintmargin','concentration','roa','roe','costinc','zscore','inslife',
          'insnonlife','stmktcap','stvaltraded','stturnover','listco_pc','prbond','pubond','intldebt','intldebtnet','nrbloan',
          'offdep','remit']
regions = ['High-income OECD members','High-income nonOECD members','Middle East and North Africa',
           'East Asia and Pacific','Latin America & the Caribbean','Europe and Central Asia','South Asia',
           'Sub-Saharan Africa']

incLevel = ['Low-income economies','Upper-middle-income economies','High-income nonOECD members',
            'Lower-middle-income economies','High-income OECD members']
for f in fields:
    for r in regions:
        noNaN = df[(df.region == r)][f] != '1.79769313486232e+308'
        media = pd.to_numeric(df[(df.region == r)][noNaN][f], downcast='float').mean()
        mediana = pd.to_numeric(df[(df.region == r)][noNaN][f], downcast='float').median()
        dif = abs(media - mediana)
        if dif >= media/4:
            df.loc[(df[f] == '1.79769313486232e+308') & (df['region'] == r), f] = mediana
        if dif < media/4:
            df.loc[(df[f] == '1.79769313486232e+308') & (df['region'] == r), f] = media

for l in incLevel:
    noNaN = df[df.incgr == l]['prbond'] != '1.79769313486232e+308'
    media = pd.to_numeric(df[df.incgr == l][noNaN]['prbond'], downcast='float').mean()
    mediana = pd.to_numeric(df[df.incgr == l][noNaN]['prbond'], downcast='float').median()
    dif = abs(media - mediana)

    for i in range(len(df.loc[(df['prbond'] == '1.79769313486232e+308') & (df['region'] == 'Middle East and North Africa') & (df.incgr == l), ['prbond']])-1):
        if dif >= media/4:
            df.loc[(df['prbond'] == '1.79769313486232e+308') & (df['region'] == 'Middle East and North Africa') & (df.incgr == l), ['prbond']] = mediana
        if dif < media/4:
            df.loc[(df['prbond'] == '1.79769313486232e+308') & (df['region'] == 'Middle East and North Africa') & (df.incgr == l), ['prbond']] = media
for f in fields:
    df[f] = pd.to_numeric(df[f], downcast='float')

# Compruebo que el dataset esta limpio y puedo trabajar

In [4]:
df.head(5)

,cn,cncode,year,region,incgr,dbacba,llgdp,cbagdp,dbagdp,ofagdp,...,stvaltraded,stturnover,listco_pc,prbond,pubond,intldebt,intldebtnet,nrbloan,offdep,remit
0,Afghanistan,AFG,1960,South Asia,Low-income economies,39.183998,36.605648,8.923634,26.232584,1.54947,...,2.93113,23.544201,0.059603,0.90993,28.129766,1.529482,0.254631,6.214133,4.39015,2.91073
1,Afghanistan,AFG,1961,South Asia,Low-income economies,30.730000,36.605648,8.923634,26.232584,1.54947,...,2.93113,23.544201,0.059603,0.90993,28.129766,1.529482,0.254631,6.214133,4.39015,2.91073
2,Afghanistan,AFG,1962,South Asia,Low-income economies,22.677000,36.605648,8.923634,26.232584,1.54947,...,2.93113,23.544201,0.059603,0.90993,28.129766,1.529482,0.254631,6.214133,4.39015,2.91073
3,Afghanistan,AFG,1963,South Asia,Low-income economies,20.194500,36.605648,8.923634,26.232584,1.54947,...,2.93113,23.544201,0.059603,0.90993,28.129766,1.529482,0.254631,6.214133,4.39015,2.91073
4,Afghanistan,AFG,1964,South Asia,Low-income economies,18.192900,36.605648,8.923634,26.232584,1.54947,...,2.93113,23.544201,0.059603,0.90993,28.129766,1.529482,0.254631,6.214133,4.39015,2.91073


In [5]:
df.corr()

,year,dbacba,llgdp,cbagdp,dbagdp,ofagdp,pcrdbgdp,pcrdbofgdp,bdgdp,fdgdp,...,stvaltraded,stturnover,listco_pc,prbond,pubond,intldebt,intldebtnet,nrbloan,offdep,remit
year,1.000000,0.076116,0.194952,0.061312,0.224940,0.057285,0.223856,0.210257,0.195495,0.192385,...,0.154148,0.099683,0.107035,0.014867,0.009730,0.149102,0.061864,0.076586,0.049118,0.148099
dbacba,0.076116,1.000000,0.402323,-0.535801,0.481873,0.267096,0.471810,0.478887,0.378765,0.384928,...,0.211263,0.256323,0.279875,0.202311,-0.179448,0.190950,0.141308,-0.018040,-0.115819,-0.040358
llgdp,0.194952,0.402323,1.000000,-0.062122,0.836486,0.423069,0.813847,0.805418,0.923096,0.930253,...,0.406688,0.337528,0.474432,0.337429,0.024566,0.387668,0.263840,0.062528,-0.027128,-0.016906
cbagdp,0.061312,-0.535801,-0.062122,1.000000,-0.135188,-0.069008,-0.162709,-0.161142,-0.089475,-0.090775,...,-0.086579,-0.098592,-0.128942,-0.069251,0.212634,-0.067345,-0.032041,0.073772,0.526194,0.074950
dbagdp,0.224940,0.481873,0.836486,-0.135188,1.000000,0.468744,0.957787,0.930682,0.761689,0.765097,...,0.462093,0.438832,0.396493,0.450311,-0.025719,0.458866,0.257400,0.049385,-0.032326,-0.083591
ofagdp,0.057285,0.267096,0.423069,-0.069008,0.468744,1.000000,0.471362,0.582830,0.389429,0.384429,...,0.454603,0.493380,0.219876,0.614793,0.191434,0.317094,0.142458,0.005483,-0.022259,-0.131056
pcrdbgdp,0.223856,0.471810,0.813847,-0.162709,0.957787,0.471362,1.000000,0.964649,0.749096,0.752662,...,0.475720,0.431195,0.420255,0.488245,-0.073374,0.507135,0.263387,0.050779,-0.026105,-0.102904
pcrdbofgdp,0.210257,0.478887,0.805418,-0.161142,0.930682,0.582830,0.964649,1.000000,0.736851,0.740350,...,0.510078,0.462544,0.421443,0.515654,-0.055813,0.481213,0.258990,0.045757,-0.028130,-0.112757
bdgdp,0.195495,0.378765,0.923096,-0.089475,0.761689,0.389429,0.749096,0.736851,1.000000,0.991436,...,0.356110,0.270551,0.451527,0.326506,-0.027341,0.446858,0.287482,0.079659,-0.020883,-0.020340
fdgdp,0.192385,0.384928,0.930253,-0.090775,0.765097,0.384429,0.752662,0.740350,0.991436,1.000000,...,0.354625,0.269936,0.456501,0.325718,-0.025839,0.447443,0.291379,0.080271,-0.021071,-0.022478


# Tras confirmar que todo esta ok, guardo una copia en nuevo csv para tener una base de trabajo sin alterar el original

In [6]:
df.to_csv(r'C:\Users\Victor\Desktop\EDA_Victor_Bandin\src\data\df_EDA_Victor_Bandin.csv', index=False, header=True)